# Демонстрация построения каскада моделей с использованием только покрытых выборок
# !!!Выборка с данными для запуска скрипта не предоставляется

## 1. Загрузка библиотек и выборок

In [1]:
import pandas as pd
import numpy as np
from vtb_scorekit.data import DataSamples
from vtb_scorekit.woe import WOE
from vtb_scorekit.model import LogisticRegressionModel
from vtb_scorekit.cascade import Cascade

### для запуска скрипта необходимо использовать свою выборку 
df = ...
module_features = ...
###

special_bins = {'vip': -999999999999}
df.loc[df['vip'] == 1, module_features['PRAVO']] = special_bins['vip']              # спец бин для ВИП сегмента в модуле PRAVO
df['year'] = df['report_date'].astype('str').str[:4]
df['pravo_hit'] = (~df[module_features['PRAVO']].isna()).all(axis=1).astype('int8') # генерим флаг покрытия факторами ПРАВО, для покрытия ФО уже есть флаг fo_hit
result_folder = 'PVR2'
df   


,calc_hub_id,inn,slx,ead,off_bal_ead,on_bal_ead,tot_debt,scale_rwa,train_flg,def_flg,...,max_segment_calc_date,inn_corr,slx_corr,id_corr_flg,calculation_source,processed_dt,report_date,report_dt,year,pravo_hit
0,None,3503003564,L6UJ9A005CR2,0.540,None,None,None,None,1,0,...,None,3503003564,L6UJ9A005CR2,no_corr,RETRO,None,2019-07-31,2019-07-31,2019,1
1,None,6625021894,L6UJ9A000EO8,0.448,None,None,None,None,0,0,...,None,6625021894,L6UJ9A000EO8,no_corr,RETRO,None,2019-07-31,2019-07-31,2019,1
2,None,2636041493,L6UJ9A002Q0R,0.375,None,None,None,None,1,0,...,None,2636041493,L6UJ9A002Q0R,no_corr,RETRO,None,2019-07-31,2019-07-31,2019,1
3,None,5041023341,L6UJ9A0029WD,0.144,None,None,None,None,1,0,...,None,5041023341,L6UJ9A0029WD,no_corr,RETRO,None,2019-07-31,2019-07-31,2019,1
4,None,3506003682,L6UJ9A002LSG,0.188,None,None,None,None,1,0,...,None,3506003682,L6UJ9A002LSG,no_corr,RETRO,None,2019-07-31,2019-07-31,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216925,None,5402483815,L6UJ9A010AZO,0.950,None,None,None,None,0,0,...,None,5402483815,L6UJ9A010AZO,no_corr,RETRO,None,2020-09-30,2020-09-30,2020,1
216926,None,7460038503,L6UJ9A00Z17P,1.405,None,None,None,None,0,0,...,None,7460038503,L6UJ9A00Z17P,no_corr,RETRO,None,2020-09-30,2020-09-30,2020,0
216927,None,212701015339,L6UJ9A00FAQ6,0.504,None,None,None,None,1,0,...,None,212701015339,L6UJ9A00FAQ6,no_corr,RETRO,None,2020-09-30,2020-09-30,2020,0
216928,None,7718852163,L6UJ9A0060SK,22.755,None,None,None,None,1,0,...,None,7718852163,L6UJ9A0060SK,no_corr,RETRO,None,2020-09-30,2020-09-30,2020,1


## 2. Создание объектов DataSamples для модулей FO, PRAVO, покрытой части выборки и полной выборки
ds - словарь вида {название модуля: DataSamples()}

In [2]:
ds = {'FO': DataSamples(samples={'train': df[df['fo_hit'] == 1]},                     # выборка для разработки. Задается в виде словаря {название_сэмпла: датафрейм}, может содержать любое кол-во сэмплов     
                        target='def_flg',                                             # целевая переменная
                        features=module_features['FO'],                               # список переменных. При None берутся все поля числового типа и нечисловые (кроме target, time_column, id_column, weights) с кол-вом уникльных значений меньше min_nunique
                        id_column='inn',                                              # уникальный в рамках среза айди наблюдения
                        time_column='year',                                           # дата среза
                        feature_descriptions=pd.read_excel('data/PVR_description.xlsx', index_col=0), # датафрейм с описанием переменных. Должен содержать индекс с названием переменных и любое кол-во полей с описанием, которые будут подтягиваться в отчеты
                        result_folder=f'{result_folder}/FO',                          # папка, в которую будут сохраняться все результаты работы с этим ДатаСэмплом
                        n_jobs=4,                                                     # кол-во используемых рабочих процессов, при -1 берется число, равное CPU_LIMIT
                        random_state=0,                                               # сид для генератора случайных чисел, используется во всех остальных методах, где необходимо
                        samples_split={},                                             # словарь с параметрами для вызова метода self.samples_split
                        ),
      
   'PRAVO': DataSamples(samples={'train': df[df['pravo_hit'] == 1]},                  # выборка для разработки. Задается в виде словаря {название_сэмпла: датафрейм}, может содержать любое кол-во сэмплов     
                        target='def_flg',                                             # целевая переменная
                        features=module_features['PRAVO'],                            # список переменных. При None берутся все поля числового типа и нечисловые (кроме target, time_column, id_column, weights) с кол-вом уникльных значений меньше min_nunique
                        id_column='inn',                                              # уникальный в рамках среза айди наблюдения
                        time_column='year',                                           # дата среза
                        feature_descriptions=pd.read_excel('data/PVR_description.xlsx', index_col=0), # датафрейм с описанием переменных. Должен содержать индекс с названием переменных и любое кол-во полей с описанием, которые будут подтягиваться в отчеты
                        result_folder=f'{result_folder}/PRAVO',                       # папка, в которую будут сохраняться все результаты работы с этим ДатаСэмплом
                        n_jobs=4,                                                     # кол-во используемых рабочих процессов, при -1 берется число, равное CPU_LIMIT
                        random_state=0,                                               # сид для генератора случайных чисел, используется во всех остальных методах, где необходимо
                        samples_split={},                                             # словарь с параметрами для вызова метода self.samples_split
                        ), 
      
    'integral_hit': DataSamples(samples={'train': df[(df['fo_hit'] == 1) & (df['pravo_hit'] == 1)]},  # выборка для разработки. Задается в виде словаря {название_сэмпла: датафрейм}, может содержать любое кол-во сэмплов     
                        target='def_flg',                                             # целевая переменная
                        features=[],                                                  # список переменных. При None берутся все поля числового типа и нечисловые (кроме target, time_column, id_column, weights) с кол-вом уникльных значений меньше min_nunique
                        id_column='inn',                                              # уникальный в рамках среза айди наблюдения
                        time_column='year',                                           # дата среза
                        result_folder=f'{result_folder}/integral',                    # папка, в которую будут сохраняться все результаты работы с этим ДатаСэмплом
                        n_jobs=4,                                                     # кол-во используемых рабочих процессов, при -1 берется число, равное CPU_LIMIT
                        random_state=0,                                               # сид для генератора случайных чисел, используется во всех остальных методах, где необходимо
                        samples_split={},                                             # словарь с параметрами для вызова метода self.samples_split
                        ), 
      
   'integral_full': DataSamples(samples={'train': df},                                # выборка для разработки. Задается в виде словаря {название_сэмпла: датафрейм}, может содержать любое кол-во сэмплов     
                    target='def_flg',                                                 # целевая переменная
                    features=[],                                                      # список переменных. При None берутся все поля числового типа и нечисловые (кроме target, time_column, id_column, weights) с кол-вом уникльных значений меньше min_nunique
                    id_column='inn',                                                  # уникальный в рамках среза айди наблюдения
                    time_column='year',                                               # дата среза
                    result_folder=f'{result_folder}/integral',                        # папка, в которую будут сохраняться все результаты работы с этим ДатаСэмплом
                    n_jobs=4,                                                         # кол-во используемых рабочих процессов, при -1 берется число, равное CPU_LIMIT
                    random_state=0,                                                   # сид для генератора случайных чисел, используется во всех остальных методах, где необходимо
                    samples_split={},                                                 # словарь с параметрами для вызова метода self.samples_split
                    ), 

     }

[INFO] [2023-05-31 12:07:17] ---------------------------------------------------------------- Creating DataSamples ----------------------------------------------------------------
[INFO] [2023-05-31 12:07:17] Selected 0 categorical features: []
[INFO] [2023-05-31 12:07:17] Actual parts of samples after samples split:
      train  Test
part    0.7   0.3
[INFO] [2023-05-31 12:07:18] DataSamples stats:
                   train         Test
amount            117282        50264
target              2873         1231
target_rate        0.024        0.024
period       2013 - 2021  2013 - 2021
[INFO] [2023-05-31 12:07:18] ---------------------------------------------------------------- Creating DataSamples ----------------------------------------------------------------
[INFO] [2023-05-31 12:07:18] Selected 0 categorical features: []
[INFO] [2023-05-31 12:07:19] Actual parts of samples after samples split:
      train  Test
part    0.7   0.3
[INFO] [2023-05-31 12:07:19] DataSamples stats:
    

## 3. Биннинг и обучение моделей модулей FO и PRAVO на заданном списке факторов

In [3]:
binning_FO = WOE(ds['FO'])
binning_FO.auto_fit(plot_flag=-1)
logreg_FO = LogisticRegressionModel(ds=binning_FO.transform(ds['FO']), transformer=binning_FO, name='FO')
logreg_FO.fit()

binning_PRAVO = WOE(ds['PRAVO'])
binning_PRAVO.auto_fit(plot_flag=-1)
logreg_PRAVO = LogisticRegressionModel(ds=binning_PRAVO.transform(ds['PRAVO']), transformer=binning_PRAVO, name='PRAVO')
logreg_PRAVO.fit()

[INFO] [2023-05-31 12:07:21] ------------------------------------------------------------------------ SFA -------------------------------------------------------------------------
[INFO] [2023-05-31 12:07:22] Performing autobinning with parameters space of size 1...
[INFO] [2023-05-31 12:07:22] Using parameters set 1/1: {'method': 'opt', 'max_n_bins': 10, 'min_bin_size': 0.05, 'criterion': 'entropy', 'scoring': 'neg_log_loss', 'max_depth': 5, 'solver': 'cp', 'divergence': 'iv', 'WOEM_on': True, 'WOEM_woe_threshold': 0.05, 'WOEM_with_missing': False, 'SM_on': False, 'SM_target_threshold': 5, 'SM_size_threshold': 100, 'G_on': False, 'G_gini_threshold': 5, 'G_gini_decrease_threshold': 0.2, 'G_gini_increase_restrict': True, 'G_with_test': False, 'WOEO_on': False, 'WOEO_dr_threshold': 0.01, 'WOEO_correct_threshold': 0.85, 'WOEO_miss_is_incorrect': True, 'WOEO_with_test': False, 'BL_on': True, 'BL_allow_Vlogic_to_increase_gini': 15, 'verbose': False}
[INFO] [2023-05-31 12:07:22] Processing 9

100%|██████████| 9/9 [00:02<00:00,  3.14it/s]


[INFO] [2023-05-31 12:07:25] Scorecard saved to the file PVR2/FO/auto_fit_scorecard.xlsx
[INFO] [2023-05-31 12:07:25] All done! 9/9 features successfully binned.
[INFO] [2023-05-31 12:07:26] Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
[INFO] [2023-05-31 12:07:26] intercept = -3.624
[INFO] [2023-05-31 12:07:26] coefs = {'f64_WOE': -0.154, 'f114_WOE': -0.563, 'f197_v3_WOE': -0.401, 'f143_WOE': -0.374, 'qn66_cur_acc_turnover_p_WOE': -0.215, 'qn56_q106_cur_ratio1_WOE': -0.132, 'f93_WOE': -0.35, 'f40_WOE': -0.291, 'f194_WOE': -0.551}
[INFO] [2023-05-31 12:07:26] ------------------------------------------------------------------------ SFA -------------------------------------------------------------------------
[INFO] [2023-05-31 12:07:26] Performing autobinning with parameters space of size 1...
[INFO] [2023-05-31 12:07:26] Using parameters set 1/1: {'method': 'opt', 'max_n_bins': 10, 'min_bin_size': 0.05, 'criterion': 'entropy', 'scoring': 'neg_log_los

100%|██████████| 4/4 [00:03<00:00,  1.23it/s]

[INFO] [2023-05-31 12:07:30] Scorecard saved to the file PVR2/PRAVO/auto_fit_scorecard.xlsx
[INFO] [2023-05-31 12:07:30] All done! 4/4 features successfully binned.


[INFO] [2023-05-31 12:07:30] Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
[INFO] [2023-05-31 12:07:30] intercept = -3.654
[INFO] [2023-05-31 12:07:30] coefs = {'act_case_otvt_dbtr_amt_p1250_WOE': -0.654, 'exec_proc_all_e1_amt_12m_p2110_WOE': -0.422, 'exec_proc_all_e1_amt_p1250_WOE': -0.237, 'case_otvt_amt_1y_p2200_p1150_WOE': -0.183}


## 4. Создание и обучение каскада моделей для покрытой выборки
Каскад состоит из двух модулей FO и PRAVO

In [4]:
cascade_hit = Cascade(models=[                         # список моделей в каскаде. Элементами списка могут быть объекты класса Cascade, LogisticRegressionModel и названия полей отдельных скоров
                              logreg_FO,               # уже обученная модель FO
                              logreg_PRAVO,            # уже обученная модель PRAVO
                              ], 
                  integral=None,                       # функция, вычисляющая интегральный скор каскада по списку скоров входящих в него моделей. При None интегральный скор вычисляется логрегом
                  ds=ds['integral_hit'],               # ДатаСэмпл, на котором будет рассчитываться интегральный скор 
                  name='integral_hit'                  # название каскада
                 )
cascade_hit.fit()

Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
[INFO] [2023-05-31 12:07:32] intercept = 2.359
[INFO] [2023-05-31 12:07:32] coefs = {'FO_score': 0.919, 'PRAVO_score': 0.727}


## 5. Создание и обучение каскада моделей для всей выборки
Каскад состоит из двух модулей FO и PRAVO, интегральной модели и флагов покрытия

In [5]:
def calc_score(scores): # scores - список [score_0, score_1, ...], где
                        #          score_i - скор модели i каскада (self.models[i]), имеет тип pd.Series
                        # в этом кейсе scores = [score_FO, score_PRAVO, score_cascade_hit, fo_hit, pravo_hit]
    score_FO, score_PRAVO, score_cascade_hit, fo_hit, pravo_hit = scores
    final_score = np.where((fo_hit == 1) & (pravo_hit == 1), score_cascade_hit,
                           np.where(fo_hit == 1, score_FO,
                                    np.where(pravo_hit == 1, score_PRAVO, 
                                             np.nan)))
    return final_score
                
cascade_full = Cascade(models=[                       # список моделей в каскаде. Элементами списка могут быть объекты класса Cascade, LogisticRegressionModel и названия полей отдельных скоров
                               logreg_FO,                                              # уже обученная вручную модель FO
                               logreg_PRAVO,  # инициализированная модель PRAVO
                               cascade_hit,
                               'fo_hit',
                               'pravo_hit' 
                             ], 
                  integral=calc_score,                # функция, вычисляющая интегральный скор каскада по списку скоров входящих в него моделей. При None интегральный скор вычисляется логрегом
                  ds=ds['integral_full'],                  # ДатаСэмпл, на котором будет рассчитываться интегральный скор 
                  name='integral_full'                 # название каскада
                 )
cascade_full.auto_logreg(ds=None,                          # ДатаСэмпл, на котором будет рассчитываться интегральный скор. При None берется self.ds
                    validate=False,                   # флаг для выполнения валидацонных тестов
                    out='auto_cascade_full.xlsx',          # либо строка с названием эксель файла, либо объект pd.ExcelWriter для сохранения отчета
                    save_model='auto_cascade_full.json'    # название json файла для сохранения каскада
                   )


------------------------------------------------------------------------------------- FO --------------------------------------------------------------------------------------

[INFO] [2023-05-31 12:07:32] Using existing self.transformer.
[INFO] [2023-05-31 12:07:32] Using existing self.coefs.

------------------------------------------------------------------------------------ PRAVO ------------------------------------------------------------------------------------

[INFO] [2023-05-31 12:08:11] Using existing self.transformer.
[INFO] [2023-05-31 12:08:11] Using existing self.coefs.

--------------------------------------------------------------------------------------------- integral_hit ---------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------- FO --------------------------------------------------------------------------------------

[INFO] [2023-05-31 12:08:38

## 6. Применение модели
### 6.1. С использованием библиотеки vtb_scorekit

In [15]:
cascade_full.scoring(ds['integral_full'],              # ДатаСэмпл или ДатаФрейм. Возвращается объект того же типа
                     score_field='score',              # поле, в которое должен быть записан посчитанный скор
                     pd_field='prob',                  # поле, в которое должен быть записан посчитанный PD
                     scale_field='scale'               # поле, в которое должен быть записан посчитанный грейд
                     ).to_df()

,calc_hub_id,inn,slx,ead,off_bal_ead,on_bal_ead,tot_debt,scale_rwa,train_flg,def_flg,...,f64_WOE,f114_WOE,FO_score,act_case_otvt_dbtr_amt_p1250_WOE,case_otvt_amt_1y_p2200_p1150_WOE,exec_proc_all_e1_amt_12m_p2110_WOE,exec_proc_all_e1_amt_p1250_WOE,PRAVO_score,integral_hit_score,sample
157063,None,5725002861,L6UJ9A001NSD,0.600,None,None,None,None,1,0,...,0.234,0.411,-5.682,0.667,0.401,0.245,0.503,-4.386,-6.052,train
73750,None,4802012625,None,0.068,None,None,None,None,0,0,...,-0.223,-0.422,-1.910,-1.530,-1.184,-1.508,-1.180,-1.521,-0.502,train
91846,None,7703701010,L6UJ9A003B56,41.184,None,None,None,None,0,0,...,0.027,-0.963,-2.278,0.667,0.401,-1.508,-1.180,-3.248,-2.095,train
168266,None,3420009153,L6UJ9A0015JU,0.102,None,None,None,None,1,0,...,0.234,-0.963,-4.840,-1.530,-0.802,-1.508,-1.180,-1.591,-3.245,train
199586,None,7726521038,L6UJ9A006CDE,0.100,None,None,None,None,1,0,...,-0.727,-0.963,-1.494,0.667,0.401,-1.508,-1.180,-3.248,-1.375,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30899,None,5321125410,L6UJ9A006NOH,0.100,None,None,None,None,1,0,...,0.234,0.411,-4.139,-1.530,0.401,0.245,-1.180,-2.550,-3.299,Test
192077,None,1831125762,L0000011JE40,0.206,None,None,None,None,0,0,...,0.234,0.411,-4.040,0.667,0.401,-1.508,-0.404,-3.431,-3.848,Test
76044,None,7722701431,L00000111PQQ,0.200,None,None,None,None,0,0,...,0.234,0.411,-4.355,0.667,-0.802,0.245,0.262,-4.109,-4.631,Test
101168,None,3652009724,L6UJ9A00452M,2.394,None,None,None,None,1,0,...,0.027,-0.140,-4.628,-0.359,0.401,0.245,-0.404,-3.500,-4.439,Test


### 6.2. Генерация хардкода для использования модели без сторонних библиотек

In [16]:
cascade_full.to_py(file_name='',                 # название питоновского файла, куда должен быть сохранен код
             score_field='score',                # поле, в которое должен быть записан посчитанный скор
             pd_field='pd',                      # поле, в которое должен быть записан посчитанный PD
             scale_field=None                    # поле, в которое должен быть записан посчитанный грейд
            )

[INFO] [2023-05-31 12:16:44] import pandas as pd
import numpy as np


def scoring(df, score_field='score', pd_field='pd', scale_field=None):
    """
    Функция скоринга выборки
    Arguments:
        df: [pd.DataFrame] входной ДатаФрейм, должен содержать все нетрансформированные переменные модели
        score_field: [str] поле, в которое должен быть записан посчитанный скор
        pd_field: [str] поле, в которое должен быть записан посчитанный PD
        scale_field: [str] поле, в которое должен быть записан посчитанный грейд
    Returns:
        df: [pd.DataFrame] выходной ДатаФрейм с добавленными полями трансформированных переменных, скоров, PD и грейда
    """

    df['f64_WOE'] = np.where(df['f64'].isnull(), -0.223, 
                             np.where((df['f64'] < 0.585), -0.99, 
                                      np.where((df['f64'] >= 0.585) & (df['f64'] < 0.854), -0.727, 
                                               np.where((df['f64'] >= 0.854) & (df['f64'] < 1.386),

In [18]:
cascade = Cascade()


Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)


In [19]:
cascade.load_model(file_name=f'PVR2/integral/auto_cascade_full.json')

The cascade was loaded from file PVR2/integral/auto_cascade_full.json
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)
Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)


In [20]:
cascade.scoring(ds['integral_full'],              # ДатаСэмпл или ДатаФрейм. Возвращается объект того же типа
                     score_field='score',              # поле, в которое должен быть записан посчитанный скор
                     pd_field='prob',                  # поле, в которое должен быть записан посчитанный PD
                     scale_field='scale'               # поле, в которое должен быть записан посчитанный грейд
                     ).to_df()

,calc_hub_id,inn,slx,ead,off_bal_ead,on_bal_ead,tot_debt,scale_rwa,train_flg,def_flg,...,f64_WOE,f114_WOE,FO_score,act_case_otvt_dbtr_amt_p1250_WOE,case_otvt_amt_1y_p2200_p1150_WOE,exec_proc_all_e1_amt_12m_p2110_WOE,exec_proc_all_e1_amt_p1250_WOE,PRAVO_score,integral_hit_score,sample
157063,None,5725002861,L6UJ9A001NSD,0.600,None,None,None,None,1,0,...,0.234,0.411,-5.682,0.667,0.401,0.245,0.503,-4.386,-6.052,train
73750,None,4802012625,None,0.068,None,None,None,None,0,0,...,-0.223,-0.422,-1.910,-1.530,-1.184,-1.508,-1.180,-1.521,-0.502,train
91846,None,7703701010,L6UJ9A003B56,41.184,None,None,None,None,0,0,...,0.027,-0.963,-2.278,0.667,0.401,-1.508,-1.180,-3.248,-2.095,train
168266,None,3420009153,L6UJ9A0015JU,0.102,None,None,None,None,1,0,...,0.234,-0.963,-4.840,-1.530,-0.802,-1.508,-1.180,-1.591,-3.245,train
199586,None,7726521038,L6UJ9A006CDE,0.100,None,None,None,None,1,0,...,-0.727,-0.963,-1.494,0.667,0.401,-1.508,-1.180,-3.248,-1.375,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30899,None,5321125410,L6UJ9A006NOH,0.100,None,None,None,None,1,0,...,0.234,0.411,-4.139,-1.530,0.401,0.245,-1.180,-2.550,-3.299,Test
192077,None,1831125762,L0000011JE40,0.206,None,None,None,None,0,0,...,0.234,0.411,-4.040,0.667,0.401,-1.508,-0.404,-3.431,-3.848,Test
76044,None,7722701431,L00000111PQQ,0.200,None,None,None,None,0,0,...,0.234,0.411,-4.355,0.667,-0.802,0.245,0.262,-4.109,-4.631,Test
101168,None,3652009724,L6UJ9A00452M,2.394,None,None,None,None,1,0,...,0.027,-0.140,-4.628,-0.359,0.401,0.245,-0.404,-3.500,-4.439,Test
